RMSProp loss learning_rate = e-4

In [1]:
import pickle
import gzip
import numpy as np
import scipy
import pandas as pd
import tensorflow as tf
#import keras
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Conv1D, Conv2D, AveragePooling2D, AveragePooling1D, Flatten, MaxPooling2D, MaxPooling1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
#from keras.utils import Sequence
from tensorflow.python.keras.utils.data_utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import imagenet_utils, mobilenet_v2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import backend as K
from time import time
from tensorflow.keras import losses
from sklearn.metrics import  roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


%matplotlib inline

print(f"Numpy version: {np.__version__}")
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

Numpy version: 1.19.5
Tensorflow version: 2.7.0
Keras version: 2.7.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import warnings
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Activation
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from keras.utils.data_utils import get_file

In [4]:
TRAIN_DIR = '/content/drive/My Drive/hes-so/data/machine1_6/train/'
TEST_DIR = '/content/drive/My Drive/hes-so/data/machine1_6/test/'

In [5]:
num_epochs = 200
learning_rate = 1e-4
decay = 4e-4

In [6]:
batch_size = 16

def preprocessing(image):
    image = tf.image.random_brightness(image, 0.125)
    image = tf.image.random_saturation(image, 0.5, 1.5)
    image = tf.image.random_hue(image, 0.2)
    image = tf.image.random_contrast(image, 0.5, 1.5)
    return image

datagen = ImageDataGenerator(
                                samplewise_center = True,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rotation_range=45,
                                rescale=1./255,
                                preprocessing_function = preprocessing,
                                validation_split=0.2
                                )

test_datagen = ImageDataGenerator(samplewise_center = True, rescale=1./255)

Iteration begins here

In [7]:
acc = []
prec = []
rec = []
f1 = []

for i in range(20):
  train_generator = datagen.flow_from_directory(
      TRAIN_DIR, 
      target_size=(299, 299),  
      color_mode='rgb',
      batch_size=batch_size,
      class_mode='categorical',
      subset='training'
  )

  val_generator = datagen.flow_from_directory(
      TRAIN_DIR,
      target_size=(299, 299),  
      color_mode='rgb',
      batch_size=batch_size,
      class_mode='categorical',
      subset='validation'
  )

  test_generator = test_datagen.flow_from_directory(
                                                      TEST_DIR,
                                                      target_size=(299, 299),  
                                                      color_mode='rgb',
                                                      batch_size=batch_size,
                                                      class_mode='categorical')

  input_tensor = Input(shape=(299, 299,3))
  base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False, input_shape = (299, 299, 3), pooling='avg')

  x = base_model.output
  x = Dense(2048, activation = 'relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation = 'relu')(x)
  x = Dropout(0.2)(x)
  predictions = Dense(3, activation = 'softmax')(x)

  model = Model(inputs=base_model.input, outputs=predictions)

  for layer in base_model.layers:
      layer.trainable = False
  optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate, decay = decay)
  model.compile(optimizer=optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])


  early_stop = EarlyStopping(monitor='val_accuracy', 
                            min_delta=0, 
                            patience=20, 
                            verbose=0, 
                            mode='max', 
                            baseline=None, 
                            restore_best_weights=True)

  history = model.fit(train_generator,
                      steps_per_epoch = 149 // batch_size,
                      epochs = 200,
                      validation_data = val_generator,
                      validation_steps = 35 // batch_size,
                      callbacks = early_stop,
                      verbose = 0
                    )

  batch_size = 1
  test_datagen = ImageDataGenerator(samplewise_center = True, rescale=1./255)
  test_generator = test_datagen.flow_from_directory(
                                                      '/content/drive/My Drive/hes-so/data/machine1_6/test_1/0/',
                                                      target_size=(299, 299),  
                                                      color_mode='rgb',
                                                      batch_size=batch_size,
                                                      class_mode='categorical')

  #Confution Matrix and Classification Report
  Y_pred = model.predict_generator(test_generator)
  y_pred_0 = np.argmax(Y_pred, axis=1)

  test_generator = test_datagen.flow_from_directory(
                                                      '/content/drive/My Drive/hes-so/data/machine1_6/test_1/1/',
                                                      target_size=(299, 299),  
                                                      color_mode='rgb',
                                                      batch_size=batch_size,
                                                      class_mode='categorical')

  #Confution Matrix and Classification Report
  Y_pred = model.predict_generator(test_generator)
  y_pred_1 = np.argmax(Y_pred, axis=1)


  test_generator = test_datagen.flow_from_directory(
                                                      '/content/drive/My Drive/hes-so/data/machine1_6/test_1/2/',
                                                      target_size=(299, 299),  
                                                      color_mode='rgb',
                                                      batch_size=batch_size,
                                                      class_mode='categorical')

  #Confution Matrix and Classification Report
  Y_pred = model.predict_generator(test_generator)
  y_pred_2 = np.argmax(Y_pred, axis=1)

  y_pred = np.concatenate((y_pred_0, y_pred_1, y_pred_2))

  y_zeros = np.zeros(18)
  y_ones = np.ones(21)
  y_twos = np.ones(41)*2

  y_true = np.concatenate((y_zeros, y_ones, y_twos))

  print(accuracy_score(y_true, y_pred), precision_score(y_true, y_pred, average='weighted'), recall_score(y_true, y_pred, average='weighted'), f1_score(y_true, y_pred, average='weighted'))

  acc.append(accuracy_score(y_true, y_pred))
  prec.append(precision_score(y_true, y_pred, average='weighted'))
  rec.append(recall_score(y_true, y_pred, average='weighted'))
  f1.append(f1_score(y_true, y_pred, average='weighted'))

Found 149 images belonging to 3 classes.
Found 35 images belonging to 3 classes.
Found 80 images belonging to 3 classes.
Found 18 images belonging to 1 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


Found 21 images belonging to 1 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


Found 41 images belonging to 1 classes.


...

Found 21 images belonging to 1 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


Found 41 images belonging to 1 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


0.7375 0.7798658578263842 0.7375 0.7460296709753231


In [8]:
np.mean(acc), np.std(acc), np.max(acc), np.min(acc)

(0.7256250000000001, 0.03546014488126071, 0.775, 0.6375)

In [9]:
np.mean(prec), np.std(prec), np.max(prec), np.min(prec)

(0.7616944772211452,
 0.027040021088206575,
 0.8292307692307691,
 0.7289044622425629)

In [10]:
np.mean(rec), np.std(rec), np.max(rec), np.min(rec)

(0.7256250000000001, 0.03546014488126071, 0.775, 0.6375)

In [11]:
np.mean(f1), np.std(f1), np.max(f1), np.min(f1)

(0.7295653494841732,
 0.03257689542856478,
 0.7751898734177215,
 0.6484577221742882)